## LGBM - ENTRENAR UN ÚNICO MODELO - FORECAST POR RECURSIVIDAD NIXTLA
## GENERAR MÁS VARIABLES EXÓGENAS

Por ejemplo:

- contador cuántos días faltan para un feriado
- contador cuántos días han pasado de un feriado


OBS IMPORTANTE: LA SEPARACIÓN DE TRAIN Y TEST SE HACE CASI AL FINAL. PORQUE CREAR VARIABLES EXÓGENAS SE PUEDE HACER UNA VEZ Y LUEGO SEPARAR, ADEMÁS CREAR VARIABLES BASADAS EN LAG NECESITA QUE EL TEST TOME ALGUNOS LAGS DE TRAIN

In [1]:
import os
# set path root of repo
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('/')[:-1]
root_path = '/'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  /Users/joseortega/Documents/GitHub/forecasting-m5-dataset


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

from scipy.stats import skew, kurtosis, variation
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor

import mlforecast
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from utilsforecast.feature_engineering import fourier


# documentacion lag transform
# https://nixtlaverse.nixtla.io/coreforecast/lag_transforms
from mlforecast.lag_transforms import RollingMean, SeasonalRollingMean
from mlforecast.lag_transforms import RollingStd, RollingMin, RollingMax
from mlforecast.lag_transforms import ExpandingMean, ExpandingStd

In [3]:
import warnings
warnings.filterwarnings("ignore")

### 1. Definir parámetros
- Recordar que las fechas se muestran como "d_xx", "date" y "week"
- Las fechas "d_xx" se pueden omitir ya que cruzando con la tabla calendario se puede obtener el "date"
- LAS FECHAS DE INICIO Y FIN DE TEST, SE OBTIENEN DEL NOTEBOOK "generar_data_test". Se define primero test y luego contar hacia atrás cuántos días mostrar

In [42]:
# TEST
date_start_test = '2016-04-25'
date_end_test = '2016-05-22'

#week_start_test = 11613
#week_end_test = 11617

In [43]:
# TRAIN. EN TRAIN SE DEFINEN LA ÚLTIMA FECHA DE ENTENAMIENTO Y CUÁNTOS DÍAS HACIA ATRÁS SE USARAN PARA EL INICIO DE TRAIN
date_end_train = '2016-04-24'
days_used_to_train = 400 

### 2. Obtener fecha de incio de entrenamiento
Desde fecha de fin y días hacia atrás a considerar para entrenamiento

In [44]:
date_end_train_datetime = pd.to_datetime(date_end_train)
date_start_train_datetime = date_end_train_datetime - pd.Timedelta(days=days_used_to_train)
date_start_train = date_start_train_datetime.strftime('%Y-%m-%d')

In [45]:
# print fecha de incio train
date_start_train

'2015-03-21'

### 2. Read raw files

In [8]:
folder_data = 'data/data_input_dtype/'

df_calender = pd.read_pickle(folder_data + 'calendar.pkl')
df_prices = pd.read_pickle(folder_data + 'sell_prices.pkl')
df_sales = pd.read_pickle(folder_data + 'sales_train_evaluation.pkl')
df_sample_output = pd.read_pickle(folder_data + 'sample_submission.pkl')

In [9]:
df_calender.head(3)

,date,wm_yr_wk,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,d_1,nan,nan,nan,nan,0,0,0
1,2011-01-30,11101,d_2,nan,nan,nan,nan,0,0,0
2,2011-01-31,11101,d_3,nan,nan,nan,nan,0,0,0


In [10]:
df_prices.head(3)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26


In [11]:
df_sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0


### 3. Transformar df_sales a formato compatible con modelos de forecast timeseries y que permita hacer merge con df_calender y df_prices

In [12]:
# transformar las ventas a formato que necesita nixtla. FORMATO: (ID_SERIE, TIMESTAMP, VALUE)
data = df_sales.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='y'
)

# correguir id de la serie. Eliminar "evaluation" para tener nombres más cortos
data['id'] = data['id'].str.rsplit('_', n=1).str[0].astype('category')

# cambiar tipo de dato a tipo category
# data['d'] = data['d'].astype(df_calender.d.dtype)

In [13]:
data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,y
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0


### 4. Merge data con df_calender y df_prices.
Tener un solo df con todas las variables exógenas. Luego, hacer algun procesamiento adicional

In [14]:
# Merge con df calender Tener df con la data de info de feriados, eventos, etc
data = data.merge(df_calender, on=['d'])

# Merge con df prices. Variable exógenas. Precios cada id. Agregación semanal
data = data.merge(df_prices, on=['store_id', 'item_id', 'wm_yr_wk'])

In [15]:
# cambiar nombre columna de fechas "date" a "ds" para replicar formato usado por otras librerias
data = data.rename(columns = {'date':'ds'})

In [16]:
# REORDENAR LAS COLUMNAS DE FORMA ALEATORIA 

# - DIFERENCIA CON RESPECTO A BASELINE. EN ESTA PRUEBA NO SE REORDENAN 

# data = data.sample(frac=1.0, random_state=0)

In [17]:
data.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,ds,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_008_CA_1,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12.0,2011-01-29,11101,nan,nan,nan,nan,0,0,0,0.46
1,HOBBIES_1_009_CA_1,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2.0,2011-01-29,11101,nan,nan,nan,nan,0,0,0,1.56
2,HOBBIES_1_010_CA_1,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,2011-01-29,11101,nan,nan,nan,nan,0,0,0,3.17


### 5. Ordenar data

In [18]:
# ordenar data de la forma [Serie, timestamp]. Ver en el dataframe los datos de cada serie ordenados por fechas
data = data.sort_values(['id', 'ds'])
data = data.reset_index().drop(columns = 'index')

In [19]:
data.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,ds,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,FOODS_1_001_CA_1,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3.0,2011-01-29,11101,nan,nan,nan,nan,0,0,0,2.0
1,FOODS_1_001_CA_1,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0.0,2011-01-30,11101,nan,nan,nan,nan,0,0,0,2.0
2,FOODS_1_001_CA_1,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0.0,2011-01-31,11101,nan,nan,nan,nan,0,0,0,2.0


### 6. Generar variables exógenas
Generar listado variables exógenas. Luego de tener un df todas las variables exógenas, separar en train y test

In [20]:
data.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,ds,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,FOODS_1_001_CA_1,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3.0,2011-01-29,11101,nan,nan,nan,nan,0,0,0,2.0
1,FOODS_1_001_CA_1,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_2,0.0,2011-01-30,11101,nan,nan,nan,nan,0,0,0,2.0
2,FOODS_1_001_CA_1,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_3,0.0,2011-01-31,11101,nan,nan,nan,nan,0,0,0,2.0


In [21]:
### crear variable exógena indicador de la serie
def create_exogena_id_serie(df):
    """ crear variable exógena indicador de la serie """
    df['id_serie'] = pd.factorize(df['id'])[0]
    return df
    
data = create_exogena_id_serie(df = data)

In [22]:
### transformar NULL de columnas eventos a un valor NO NULL para ser aceptado por modelos. Se transforma a "nan"
# LO MÁS LÓGICO ES TRANSFORMAR A NÚMEROS, PERO LOS MODELOS DE ÁRBOLES PUEDEN MANEJAR LAS VARIABLES STRING, ASI QUE NO ES NECESARIO

# data calender ya viene de esta forma

In [23]:
### agregar variable exógena binaria - 1: es un dia feriado por calendario // 0: no lo es
def create_exogena_binaria_event_calender(df):
    """ agregar variable exógena binaria 1: es un día de evento por calendario // 0: no lo es """
    
    # obs en una de las transformaciones de los datos, los NaN se dejaron como string 'nan'
    # si al menos una de las columnas 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2' tiene valor, ES UN EVENTO
    df['event'] = df[['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']].replace('nan', np.NaN).notnull().any(axis=1).astype(int)

    return df
    
data = create_exogena_binaria_event_calender(df = data)

In [24]:
# MOSTRAR LOS EVENTOS DE CADA AÑO
# data[data['event'] == 1].groupby(['ds'])[['ds', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'event']].first()

In [25]:
### agregar variabler exógena contador de todos los días antes y después de algún evento. Que diga 1 días del evento, 2 días antes, etc
def create_exogena_contador_dias_antes_y_despues_event(df):
    """ 
    Agregar variabler exógena contador de todos los días antes y después de algún evento. Que diga 1 días del evento, 2 días antes, etc
    - Se cuantan los días que faltan para el próximo evento
    - Se cuentan los días que han pasado desde el último evento
    """
    df_ds_oneserie = df[['ds', 'event']].drop_duplicates()
    df_ds_oneserie.sort_values(by = 'ds', ascending = True, inplace = True)

    holidays_aux = df_ds_oneserie['ds'].loc[df_ds_oneserie['event'] == 1].tolist()
    holidays_aux = pd.to_datetime(holidays_aux, dayfirst=True)
    df_ds_oneserie_hd = pd.DataFrame({'date1':holidays_aux})

    out = pd.merge_asof(df_ds_oneserie, df_ds_oneserie_hd, left_on='ds', right_on='date1', direction='forward')
    out = pd.merge_asof(out, df_ds_oneserie_hd, left_on='ds', right_on='date1')

    out['days_last_holiday'] = out['ds'].sub(out.pop('date1_y')).dt.days
    out['days_next_holiday'] = out.pop('date1_x').sub(out['ds']).dt.days

    df = pd.merge(df, out, on=['ds', 'event'], how='left')
    df['days_last_holiday'] = df['days_last_holiday'].fillna(0)
    df['days_next_holiday'] = df['days_next_holiday'].fillna(0) # rellenar nulos con cero. DE QUÉ OTRA FORMA SE PODRIAN RELLENAR LOS NULOS
    
    return df


data = create_exogena_contador_dias_antes_y_despues_event(df = data)

In [26]:
### crear indicadores si es día de semana, si es fin de semana, si es sabado, si es domingo
def create_exogena_indicadores_dias_semana(df):
    """ crear indicadores si es día de semana, si es fin de semana, si es sabado, si es domingo """
    
    # si es dia de semana
    df['week_day'] = (df['ds'].dt.dayofweek <= 4).astype(int)

    # si es sabado
    #df['sabado_day'] = (df['ds'].dt.dayofweek == 5).astype(int)

    # si es domingo
    #df['domingo_day'] = (df['ds'].dt.dayofweek == 6).astype(int)

    # indicador si es la primera, segunda, tercera, cuarta semana del mes
    df['indice_semana_en_mes'] = (df['ds'].dt.day - 1) // 7 + 1
    
    return df


data = create_exogena_indicadores_dias_semana(df = data)

In [30]:
### calcular variacion en precios para cada serie con respecto a semanas previas

def create_exogena_variacion_prices_past_weeks(df, df_prices, list_delay):
    """
    Tomar df con precios por semana y calcular variación en el precio con respecto a semanas previas.
    Se puede hacer varias comparación con respecto a varias semanas previas
    Luego, se agrega la variación semanal en los precios
    
    IMPORTANTE: AL GENERAR VARIACIÓN SE VAN A GENERAR NULOS. REVISARLO EN PROX PASOS. puede NO SER UN PROBLEMA, si se toman
    X observaciones atrás sin considerar toda la data para entrenar
    
    Args
        df (DataFrame): Dataframe con la data
        df_prices (DataFrame): DataFrame que tiene solo los precios (estructura del df original con los precios)
        list_delay (list): Lista con semanas a calcular delay de precios por cada serie. 1: vs semana anterior, 4: vs 4 semanas atras
    Return
        df (DataFrame): DataFrame con la data, agregada las variables exógenas relacionadas a delay con precios
    """
    
    # crear variable auxiliar con precios
    df_aux_prices = df_prices.copy()


    # recorrer cada item_id y calcular la variacion de precio con respecto a un periodo anterior
    list_unique_item_id = df_aux_prices['item_id'].unique()
    for unique_item_id in list_unique_item_id:
    
        # calcular mask para item id que cuadre. 
        # PODER FILTRAR DATAFRAME SOLO POR EL ITEM ID QUE INTERESA SIN TENER QUE CREAR DF AUXLIAR
        mask_unique_item_id = df_aux_prices['item_id'] == unique_item_id

        
        for delay in list_delay:
            
            # llevar precios del pasado al futuro para poder hacer precio(t) - precio(t-1)
            df_aux_prices.loc[mask_unique_item_id, 'sell_price_lagged'] = df_aux_prices.loc[mask_unique_item_id, 'sell_price'].shift(delay)
            
            # calcular delta de precio con respecto a variación de un periodo anterior
            df_aux_prices.loc[mask_unique_item_id, f'delta_price_{delay}'] = df_aux_prices.loc[mask_unique_item_id, 'sell_price'] - df_aux_prices.loc[mask_unique_item_id, 'sell_price_lagged']

    
    # Merge variacion semanal precios con data
    df_aux_prices = df_aux_prices.drop(columns = ['sell_price', 'sell_price_lagged'])
    df = df.merge(df_aux_prices, on=['store_id', 'item_id', 'wm_yr_wk'])

    return df

In [31]:
data.shape

(46881677, 24)

In [32]:
### calcular variacion en precios para cada serie con respecto a semanas previas
data = create_exogena_variacion_prices_past_weeks(df = data, 
                                                  df_prices = df_prices, 
                                                  list_delay = [1, 4, 8, 12]
                                                 )

In [33]:
data.shape

(46881677, 28)

In [ ]:
stop

### 7. Eliminar columnas no utiizada
Eliminar columnas no utilizadas en la data

In [34]:
data.shape

(46881677, 28)

In [35]:
## PASO FINAL - ELIMINAR COLUMNAS NO UTILIZADA
# eliminar columnas
data = data.drop(columns=['d', 'wm_yr_wk'])

In [36]:
data.shape

(46881677, 26)

### 8. Filtrar data train y test
- Filtrar data train de acuerdo a rango de fechas utilizados para train
- Existe una función que consulta métrica (obtiene datos reales de test) y calcula métrica

In [50]:
################## separar data train y test de acuerdo a rango de fechas definidas ##################
mask_train = (data['ds'] >= date_start_train) & (data['ds'] <= date_end_train)
data_train = data[mask_train]

mask_test = (data['ds'] >= date_start_test) & (data['ds'] <= date_end_test)
data_test = data[mask_test]

print('shape train: ', data_train.shape)
print('shape test: ', data_test.shape)

shape train:  (12177819, 26)
shape test:  (853720, 26)


In [60]:
################## agregar variable exógena transformada de fourier ##################

def create_exogena_transformada_fourier(df_train, df_test, diferencia_dias_horizon_fcst):
    """ agregar variable exógena transformada de fourier. Se utiliza clase nixtla que recibe data train y retorna data train y data test/future """

    # renombrar columna "id" al nombre "unique_id". nixlta necesita que tenga ese nombre
    df_train = df_train.rename(columns = {'id': 'unique_id'})
    df_test = df_test.rename(columns = {'id': 'unique_id'})

    
    transformed_fourier_df_train, future_fourier_df = fourier(df_train, freq='D', season_length = 7, k = 3, h = diferencia_dias_horizon_fcst)

    df_train = transformed_fourier_df_train # funcion retorna el dataframe de train agregadas las columnas de las transformada
    df_test = pd.merge(df_test, future_fourier_df, on=['unique_id', 'ds'], how='inner') # funcion retorna solo la transformada para los datos de test - unir


    # regresar a nombre id de columnas original
    df_train = df_train.rename(columns = {'unique_id': 'id'})
    df_test = df_test.rename(columns = {'unique_id': 'id'})

    
    return df_train, df_test


data_train, data_test = create_exogena_transformada_fourier(df_train = data_train, 
                                                            df_test = data_test,
                                                            diferencia_dias_horizon_fcst = 28
                                                           )

In [61]:
data_train.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'y', 'ds',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'id_serie', 'event',
       'days_last_holiday', 'days_next_holiday', 'week_day',
       'indice_semana_en_mes', 'delta_price_1', 'delta_price_4',
       'delta_price_8', 'delta_price_12', 'sin1_7', 'sin2_7', 'sin3_7',
       'cos1_7', 'cos2_7', 'cos3_7'],
      dtype='object')

In [ ]:
################## separar data train y test de acuerdo a rango de fechas definidas ##################